In [1]:
import json
import numpy as np
from experiments import MOT16_Experiments
Settings = json.load(open('settings.txt'))
root = Settings['data_root']
mot16 = MOT16_Experiments(root)

Hy = mot16.mot16_11_true_detections
VIDEO = "MOT16-11"

X = mot16.mot16_11_X

# I = mot16.mot16_11_X[frame]

n, _ = Hy.shape
print(Hy.shape)

delta_max = 5 
delta_theta = 3

from GenerateFeatureVector import pairwise_features
gen = pairwise_features(delta_max)

pairwise_vectors = [[] for _ in range(delta_max)]
labels = [[] for _ in range(delta_max)]

for i in range(n):
    frame1, id1, x1, y1, w1, h1,conf1 = Hy[i]
    I1 = X[int(frame1-1)]
    for j in range(i+1, n):
        frame2, id2, x2, y2, w2, h2,conf2 = Hy[j]
        
        I2 = X[int(frame2-1)]
        delta = int(abs(frame2-frame1) )
        if delta >= delta_max :
            continue 
        
        pair_vec = gen.get_pairwise_vector(VIDEO ,I1,I2,frame1,frame2,(x1, y1, w1, h1),(x2, y2, w2, h2),conf1,conf2)
        

        
        assert delta < delta_max, "delta too big:" + str(delta)
        pairwise_vectors[delta].append(pair_vec)
        labels[delta].append(1 if id1==id2 else 0)
    print("detection: ",i," out of ",n)
    
        

print (Hy.shape)

# print(X, Y.T)

# w = la.lstsq(X, Y)[0]

# print(w)

#w = linregress(X, Y.T)

(3349, 7)


Using TensorFlow backend.


Found model /Users/basheq/Desktop/RGBD_Lab/Data/reid_models/stacknet64x64_84acc.h5! :)


KeyboardInterrupt: 

In [ ]:
import numpy.linalg as la

print (len(pairwise_vectors[0]))
print (len(labels[0]))

weights = []

for i in range(delta_max):
    X_ = np.array( pairwise_vectors[i])
    n, count_features = X_.shape
    X = np.ones((n, count_features+1))
    X[:,1:] = X_
    
    Y = np.array(labels[i])
    print (Y.shape)
    w = la.lstsq(X,Y)[0]
    print(i)
    print (len(w))
    print(w.shape)
    
    weights.append(w)
    
